<a href="https://colab.research.google.com/github/AugustinCablant/Viager/blob/main/Agence_unique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports


In [1]:
!pip install unidecode
!pip install dateparser
import pandas as pd
import numpy as np
import dateparser
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Stage')
import matplotlib.pyplot as plt
import re
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from dateutil import parser
from datetime import datetime
from unidecode import unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Fichiers de l'INSEE

In [2]:
dossier = '/content/drive/MyDrive/Stage/sas/sas/'
# Liste pour stocker les DataFrames
dataframes = []

# Boucle pour créer ma liste de DataFrames

for i in tqdm(range(1970,1990)):
    # Obtenir la liste des fichiers CSV dans le dossier
    chemins_fichiers_sas = dossier+f'a{i}.sas7bdat'
    dataframes.append(pd.read_sas(chemins_fichiers_sas))

df_ins = pd.concat(dataframes)
n = df_ins.shape[0]
df_ins['index']= list(range(1,n+1))
df_ins.set_index('index', inplace=True)

# Data Frame Insee avec les bons noms de colonne et les bons formats
df_insee1 = pd.DataFrame(index=list(range(1,df_ins.shape[0])), columns = ['nom', 'prenoms',
        'sexe', 'ybirth', 'mbirth', 'dbirth','villebirth', 'paysbirth', 'Dpt_naissance',
        'ydeath', 'mdeath', 'ddeath', 'codebirth', 'codebirth', 'nbdeath', 'type', 'reserve', 'fichier', 'allobs'])


df_insee1['B_nom'] = df_ins['nom'].str.decode('utf-8', 'ignore')[2:-1]
df_insee1['B_prenoms'] = df_ins['prenoms'].str.decode('utf-8', 'ignore')[2:-1]
df_insee1['B_annee'] = df_ins['ybirth']
df_insee1['B_mois'] = df_ins['mbirth']
df_insee1['B_jour'] = df_ins['dbirth']
df_insee1['D_annee'] = df_ins['ydeath']
df_insee1['D_mois'] = df_ins['mdeath']
df_insee1['D_jour'] = df_ins['ddeath']
df_insee1['B_departement'] = (df_ins['codebirth'].str.decode('utf-8')).str.slice(0,2)
df_insee1['B_pays'] = df_ins['paysbirth'].str.decode('iso-8859-1')
df_insee1['B_ville'] = df_ins['villebirth'].str.decode('iso-8859-1')
df_insee1['codebirth'] = df_ins['codebirth']
df_insee1['nbdeath'] = df_ins['nbdeath']
df_insee1['type'] = df_ins['type']
df_insee1['reserve'] = df_ins['reserve']
df_insee1['fichier'] = df_ins['fichier']
df_insee1['allobs'] = df_ins['allobs']
df_insee1['B_sexe'] = df_ins['sex']
df_insee1['B_departement']= pd.to_numeric(df_insee1['B_departement'], errors='coerce').astype(float)

def f(chaine):
  if pd.notnull(chaine):
    return chaine.split(' ')[0]
  else:
    pass

# variables de match
df_insee1['nom'] = df_insee1['B_nom']
df_insee1['prenoms'] = df_insee1['B_prenoms']
df_insee1['prenoms1'] = df_insee1['B_prenoms'].apply(f)
df_insee1['ybirth'] = df_insee1['B_annee']
df_insee1['mbirth'] = df_insee1['B_mois']
df_insee1['dbirth'] = df_insee1['B_jour']
df_insee1['sexe'] = df_insee1['B_sexe']
df_insee1['Dpt_naissance'] = df_insee1['B_departement']


100%|██████████| 20/20 [01:29<00:00,  4.46s/it]


## DATA AGENCE UNIQUE

In [8]:
df_grille = pd.read_csv('data_agence.csv', encoding = 'Latin-1')

def convert_to_timestamp(value):
    if pd.isnull(value) or value == '/':
        return value
    else:
        try:
            timestamp = dateparser.parse(str(value))
            return timestamp
        except ValueError:
            return value

df_grille['Date_naissance_vendeur_1'] = df_grille['Date_naissance_vendeur_1'].apply(convert_to_timestamp)
df_grille['Date_naissance_vendeur_2'] = df_grille['Date_naissance_vendeur_2'].apply(convert_to_timestamp)
df_grille = df_grille.dropna(subset = 'Tete_1')
df_grille['Tete_1'] = df_grille['Tete_1'].astype(str)
df_grille['Tete_2'] = df_grille['Tete_2'].astype(str)



colonnes = ['Adresse_bien_vendu', 'Prenom_vendeur_1', 'Nom_vendeur_1',
       'Nom_vendeur_2', 'Nom_acheteur_2', 'Type_de_viager', 'Majoration',
       'Bouquet', 'Rente', 'Tete_1', 'Tete_2', 'Commission', 'Milliemes',
       'Standing', 'Remarques', 'Valeur_libre', 'Valeur_occupee', 'Numero',
       'Repertoire', 'Prenom_vendeur_2', 'Date_naissance_vendeur_1',
       'Ville_naissance_vendeur_1', 'Departement_naissance_vendeur_1',
       'Pays_naissance_vendeur_1', 'Date_naissance_vendeur_2',
       'Ville_naissance_vendeur_2', 'Departement_naissance_vendeur_2',
       'Pays_naissance_vendeur_2', 'Adresse_vendeur', 'Nom_acheteur_1',
       'Profession_acheteur_1', 'Date_naissance_acheteur_1',
       'Profession_acheteur_2', 'Date_naissance_acheteur_2',
       'Adresse_acheteur', 'Indexation_rente', 'Date_acte_authentique',
       'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces', 'Etage',
       'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave', 'Date_immeuble',
       'Pierre_de_taille', 'source', 'Ch_de_service', 'Parking', 'Bonus',
       'annee', 'type', 'nom1', 'nom2', 'prenom1', 'prenom2', 'sexe1', 'sexe2',
       'annee1', 'annee2', 'naissancetete1', 'mois1', 'jour1',
       'naissancetete2', 'mois2', 'jour2', 'lieutete1', 'dep1', 'lieutete2',
       'dep2', 'abroad1', 'abroad2', 'quality', 'nom', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1',
       'ville1', 'ville2']

colonnes1 = ['nom1', 'prenom1', 'annee1', 'mois1', 'jour1',
       'dep1', 'abroad1','ville1','sexe1',
        'Adresse_bien_vendu', 'Prenom_vendeur_1', 'Nom_vendeur_1',
       'Nom_vendeur_2', 'Nom_acheteur_2', 'Type_de_viager', 'Majoration',
       'Bouquet', 'Rente', 'Tete_1', 'Tete_2', 'Commission', 'Milliemes',
       'Standing', 'Remarques', 'Valeur_libre', 'Valeur_occupee', 'Numero',
       'Repertoire', 'Prenom_vendeur_2', 'Date_naissance_vendeur_1',
       'Ville_naissance_vendeur_1', 'Departement_naissance_vendeur_1',
       'Pays_naissance_vendeur_1', 'Date_naissance_vendeur_2',
       'Ville_naissance_vendeur_2', 'Departement_naissance_vendeur_2',
       'Pays_naissance_vendeur_2', 'Adresse_vendeur', 'Nom_acheteur_1',
       'Profession_acheteur_1', 'Date_naissance_acheteur_1',
       'Profession_acheteur_2', 'Date_naissance_acheteur_2',
       'Adresse_acheteur', 'Indexation_rente', 'Date_acte_authentique',
       'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces', 'Etage',
       'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave', 'Date_immeuble',
       'Pierre_de_taille', 'source', 'Ch_de_service', 'Parking', 'Bonus',
       'annee', 'type', 'quality', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1']

colonnes2 = ['nom2', 'prenom2', 'annee2', 'mois2', 'jour2',
       'dep2', 'abroad2','ville2','sexe2',
       'Adresse_bien_vendu', 'Prenom_vendeur_1', 'Nom_vendeur_1',
       'Nom_vendeur_2', 'Nom_acheteur_2', 'Type_de_viager', 'Majoration',
       'Bouquet', 'Rente', 'Tete_1', 'Tete_2', 'Commission', 'Milliemes',
       'Standing', 'Remarques', 'Valeur_libre', 'Valeur_occupee', 'Numero',
       'Repertoire', 'Prenom_vendeur_2', 'Date_naissance_vendeur_1',
       'Ville_naissance_vendeur_1', 'Departement_naissance_vendeur_1',
       'Pays_naissance_vendeur_1', 'Date_naissance_vendeur_2',
       'Ville_naissance_vendeur_2', 'Departement_naissance_vendeur_2',
       'Pays_naissance_vendeur_2', 'Adresse_vendeur', 'Nom_acheteur_1',
       'Profession_acheteur_1', 'Date_naissance_acheteur_1',
       'Profession_acheteur_2', 'Date_naissance_acheteur_2',
       'Adresse_acheteur', 'Indexation_rente', 'Date_acte_authentique',
       'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces', 'Etage',
       'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave', 'Date_immeuble',
       'Pierre_de_taille', 'source', 'Ch_de_service', 'Parking', 'Bonus',
       'annee', 'type', 'quality', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1']

df_travail = pd.DataFrame(index = list(range(1,df_grille['nom1'].count()+df_grille['nom2'].count()+1)),
                          columns = colonnes)

df1 = df_grille[pd.notnull(df_grille['nom1'])][colonnes1]
df2 = df_grille[pd.notnull(df_grille['nom2'])][colonnes2]

df1 = df1.rename(columns={'nom1': 'A_nom', 'prenom1' : 'A_prenoms',
                          'annee1' : 'A_annee' , 'mois1' : 'A_mois',
       'jour1' : 'A_jour', 'ville1' : 'A_ville',
                          'dep1' : 'A_departement',
       'abroad1':'A_abroad', 'sexe1' : 'A_sexe'})
df2 = df2.rename(columns={'nom2': 'A_nom', 'prenom2' : 'A_prenoms',
                          'annee2' : 'A_annee' , 'mois2' : 'A_mois',
       'jour2' : 'A_jour', 'ville2' : 'A_ville',
                          'dep2' : 'A_departement',
       'abroad2':'A_abroad', 'sexe2' : 'A_sexe'})


df1['head'] = 1
df2['head'] = 2
df_travail = pd.concat([df1, df2], ignore_index = True)
df_travail['A_nom'] = df_travail['A_nom'].str.upper()
df_travail['A_prenoms'] = df_travail['A_prenoms'].str.upper()

# Remplacer les valeurs NaN par une chaîne de caractères vide
df_travail['A_prenoms'] = df_travail['A_prenoms'].fillna('')

df_travail['A_prenoms'] = df_travail['A_prenoms'].apply(unidecode)
df_travail['A-departement'] = pd.to_numeric(df_travail['A_departement'], errors='coerce').astype('Int64').astype(str)
df_travail['A_sexe'] = df_travail['A_sexe'].astype(float)
df_travail['A_nom'] = df_travail['A_nom'].str.upper()

# création des colonnes de match avec le df INSEE
df_travail['nom'] = df_travail['A_nom']
df_travail['sexe'] = df_travail['A_sexe']
df_travail['ybirth'] = df_travail['A_annee']
df_travail['mbirth'] = df_travail['A_mois']
df_travail['dbirth'] = df_travail['A_jour']
df_travail['Dpt_naissance'] = df_travail['A_departement']

#création du fichier composé des noms, prenoms, sexe, année/mois/jour de naissance
"""
unique = df_travail[df_travail.drop_duplicates(subset=['identifiant'], keep = 'first')][
    ['A_nom','A_sexe','A_annee','A_mois','A_jour','A_departement','nom','sexe',
    'ybirth','mbirth','dbirth','dpt_naissance']]

unique.to_csv('unique/unique.csv')
"""

<ipython-input-8-5702593903f7>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grille['Tete_1'] = df_grille['Tete_1'].astype(str)
<ipython-input-8-5702593903f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grille['Tete_2'] = df_grille['Tete_2'].astype(str)


"\nunique = df_travail[df_travail.drop_duplicates(subset=['identifiant'], keep = 'first')][\n    ['A_nom','A_sexe','A_annee','A_mois','A_jour','A_departement','nom','sexe',\n    'ybirth','mbirth','dbirth','dpt_naissance']]\n\nunique.to_csv('unique/unique.csv')\n"

## MERGE

In [26]:
unique = pd.read_csv('unique/imparfait_postdepartement_postprenom_postprenoms1_postnonom.csv')
def f(chaine):
  if pd.notnull(chaine):
    return chaine.split(' ')[0]
  else:
    pass
unique['A_ville'] = unique['A_ville'].str.upper()
unique['prenoms1'] = unique['A_prenoms'].apply(f)
unique[['nom','B_ville']] = unique[['A_nom','A_ville']]
merged_df = unique.merge(df_insee1, on=['nom','B_ville'], how='left')

merged_df = merged_df.dropna(subset = ['B_annee'] )

In [5]:
#merged_df = merged_df.drop([37,38,140,156,250,279,332,333,355,405,406,417])
#merged_df.to_csv('unique/imparfait_2010_2023.csv')

#merged_df= merged_df[merged_df.index.isin([6,7,24,29,40,42,47,50,66,72,77,78,121,133,180,210])]

In [35]:
merged_df[['A_prenoms','B_prenoms','A_mois','B_mois','A_jour','B_jour','A_ville','B_ville','A_annee','B_annee']][250:300]
#merged_df[merged_df['B_prenoms']=='ANDREE'][['A_prenoms','A_abroad','B_prenoms','A_ville','B_ville','A_annee','B_annee']][50:100]

,A_prenoms,B_prenoms,A_mois,B_mois,A_jour,B_jour,A_ville,B_ville,A_annee,B_annee
472,CLAUDE BERTHE LOUISE,ROBERT STEPHANE,4.0,6.0,6.0,5.0,NANTES,NANTES,1927.0,1904.0
473,CLAUDE BERTHE LOUISE,EMILE JULIEN,4.0,9.0,6.0,24.0,NANTES,NANTES,1927.0,1906.0
474,CLAUDE BERTHE LOUISE,HENRI JEAN GABRIEL MARIE,4.0,6.0,6.0,24.0,NANTES,NANTES,1927.0,1917.0
475,CLAUDE BERTHE LOUISE,PAUL ROBERT JEAN,4.0,3.0,6.0,8.0,NANTES,NANTES,1927.0,1922.0
476,CLAUDE BERTHE LOUISE,FELIX JOSEPH MARIE,4.0,2.0,6.0,1.0,NANTES,NANTES,1927.0,1905.0
477,CLAUDE BERTHE LOUISE,ANNE-SOLENNE,4.0,9.0,6.0,20.0,NANTES,NANTES,1927.0,1978.0
478,CLAUDE BERTHE LOUISE,AUGUSTE FRANCOIS MATHIEU,4.0,1.0,6.0,29.0,NANTES,NANTES,1927.0,1892.0
479,CLAUDE BERTHE LOUISE,ARMAND HENRI PIERRE,4.0,12.0,6.0,5.0,NANTES,NANTES,1927.0,1894.0
480,CLAUDE BERTHE LOUISE,JACQUES JEAN GILBERT VINCENT,4.0,4.0,6.0,13.0,NANTES,NANTES,1927.0,1931.0
481,CLAUDE BERTHE LOUISE,WILFRIED RENE JACQUES,4.0,8.0,6.0,27.0,NANTES,NANTES,1927.0,1980.0


In [5]:
"""
liste_identifiant = merged_df['identification'].to_list()
to_drop = unique[unique['identification'].isin(liste_identifiant)].index.to_list()
unique_post_parfait_70_90 = unique.drop(to_drop)
unique_post_parfait_70_90.to_csv('unique_post_parfait_70_90.csv')
"""
merged_df.to_csv('unique/imparfait_no_nom_70_90.csv')

In [6]:
df1 = pd.read_csv('unique/imparfait_no_nom_70_90.csv')
df2 = pd.read_csv('unique/imparfait_no_nom_90_2010.csv')
df3 = pd.read_csv('unique/imparfait_no_nom_2010_2023.csv')

df = pd.concat([df1,df2,df3], axis = 0, ignore_index = True)
df = df[['head','identification', 'A_nom', 'A_prenoms', 'A_annee',
       'A_mois', 'A_jour', 'A_departement', 'A_abroad', 'A_ville', 'A_sexe',
         'B_nom', 'B_prenoms', 'B_annee', 'B_mois',
       'B_jour', 'D_annee', 'D_mois', 'D_jour', 'B_departement', 'B_pays',
       'B_ville', 'B_sexe','codebirth', 'nbdeath', 'type', 'reserve',
       'fichier', 'allobs', ]]
#df.to_csv('unique_parfait.csv')

In [11]:
df.to_csv('unique/uniques/imparfait_no_nom.csv')


In [12]:
unique = pd.read_csv('unique/imparfait_postdepartement_postprenom_postprenoms1.csv')
unique_imparfait = unique.copy()
#df_parfait = pd.read_csv('unique/unique_parfait.csv')

id = df['identification'].to_list()

for i in unique_imparfait.index.to_list():
  if unique_imparfait['identification'][i] in id:
    unique_imparfait = unique_imparfait.drop(i)

unique_imparfait.to_csv('unique/imparfait_postdepartement_postprenom_postprenoms1_postnonom.csv')

## Création de l'identifiant

In [27]:
unique_imparfait['age'] = np.nan
for i in unique_imparfait.index.to_list():
  if pd.notnull(unique_imparfait['A_annee'][i]):
    unique_imparfait['age'][i] = 2023 - int(unique_imparfait['A_annee'][i])

unique_imparfait['age'].describe()

<ipython-input-27-eeb92565cb48>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_imparfait['age'][i] = 2023 - int(unique_imparfait['A_annee'][i])


count    382.000000
mean     106.290576
std        9.661563
min       82.000000
25%       99.000000
50%      106.000000
75%      113.000000
max      137.000000
Name: age, dtype: float64

In [ ]:
df_travail.sort_values(by = 'nom')
unique = df_travail.drop_duplicates(subset = ['A_nom','A_prenoms','A_annee','A_mois','A_jour','A_departement'], keep = 'first')
unique = unique[['A_nom','A_prenoms','A_annee','A_mois','A_jour','A_departement','A_abroad','A_ville','A_sexe']]
unique['identifiant'] = range(1,unique.shape[0]+1)
#unique.to_csv('unique/unique1.csv')

In [ ]:
df_travail['identifiant'] = 0
for i in unique.index.to_list():
  id = unique['identifiant'][i]
  nom = unique['A_nom'][i]
  prenoms = unique['A_prenoms'][i]
  sexe = unique['A_sexe'][i]
  annee = unique['A_annee'][i]
  mois = unique['A_mois'][i]
  jour = unique['A_jour'][i]
  departement = unique['A_departement'][i]
  for j in df_travail.index.tolist():
    if df_travail['A_nom'][j]==nom:
      if df_travail['A_prenoms'][j]==prenoms:
        df_travail['identifiant'][j] = id

df_travail = df_travail[['identifiant','A_nom', 'A_prenoms', 'A_annee', 'A_mois', 'A_jour', 'A_departement',
       'A_abroad', 'A_ville', 'A_sexe', 'Adresse_bien_vendu',
       'Prenom_vendeur_1', 'Nom_vendeur_1', 'Nom_vendeur_2', 'Nom_acheteur_2',
       'Type_de_viager', 'Majoration', 'Bouquet', 'Rente', 'Tete_1', 'Tete_2',
       'Commission', 'Milliemes', 'Standing', 'Remarques', 'Valeur_libre',
       'Valeur_occupee', 'Numero', 'Repertoire', 'Prenom_vendeur_2',
       'Date_naissance_vendeur_1', 'Ville_naissance_vendeur_1',
       'Departement_naissance_vendeur_1', 'Pays_naissance_vendeur_1',
       'Date_naissance_vendeur_2', 'Ville_naissance_vendeur_2',
       'Departement_naissance_vendeur_2', 'Pays_naissance_vendeur_2',
       'Adresse_vendeur', 'Nom_acheteur_1', 'Profession_acheteur_1',
       'Date_naissance_acheteur_1', 'Profession_acheteur_2',
       'Date_naissance_acheteur_2', 'Adresse_acheteur', 'Indexation_rente',
       'Date_acte_authentique', 'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces',
       'Etage', 'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave',
       'Date_immeuble', 'Pierre_de_taille', 'source', 'Ch_de_service',
       'Parking', 'Bonus', 'annee', 'type', 'quality', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1',
       'head', 'A-departement', 'nom', 'sexe', 'ybirth', 'mbirth', 'dbirth',
       'Dpt_naissance']]

#df_travail.to_csv('df_travail.csv')

In [73]:
unique = df_travail.drop_duplicates(subset = ['A_nom', 'A_prenoms'])
unique['identification'] = range(1,unique.shape[0]+1)
df_travail['identification'] = 0

for i in unique.index.to_list():
  identification = unique['identification'][i]
  nom = unique['A_nom'][i]
  prenom = unique['A_prenoms'][i]
  for k in df_travail.index.to_list():
    if df_travail['A_nom'][k]==nom:
      if df_travail['A_prenoms'][k]==prenom:
        df_travail['identification'][k]= identification


<ipython-input-73-fcf002c07c6a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique['identification'] = range(1,unique.shape[0]+1)
<ipython-input-73-fcf002c07c6a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_travail['identification'][k]= identification


## MERGED des df

In [36]:
df1 = pd.read_csv('unique/uniques/unique_parfait.csv')
df2 = pd.read_csv('unique/uniques/imparfait_prenoms.csv')
df3 = pd.read_csv('unique/uniques/imparfait_prenoms1.csv')
df4 = pd.read_csv('unique/uniques/imparfait_no_nom.csv')
df5 = pd.read_csv('unique/uniques/imparfait_departement.csv')

In [49]:
matching = pd.concat([df1,df2,df3,df4,df5],axis=0,ignore_index=True)

In [61]:
matching

,head,identification,A_nom,A_prenoms,A_annee,A_mois,A_jour,A_departement,A_abroad,A_ville,...,B_departement,B_pays,B_ville,B_sexe,codebirth,nbdeath,type,reserve,fichier,allobs
0,1,20,ARMAND,RENE,1897.0,10.0,31.0,16.0,0.0,Saint-Sulpice-de-Cognac,...,16.0,NaN,SAINT-SULPICE-DE-COGNAC,1.0,b'16355',b'417',b'N',NaN,1988.0,b'ARMAND*RENE/ ...
1,1,34,AUGSBURGER,ANDREE MARTHE,1910.0,5.0,30.0,75.0,0.0,Paris,...,75.0,NaN,PARIS 9,2.0,b'75109',b'227',b'N',NaN,1980.0,b'AUGSBURGER*ANDREE MARTHE/ ...
2,1,55,BARCOS,ROBERT,1907.0,9.0,22.0,75.0,NaN,NaN,...,75.0,NaN,PARIS 8E ARRONDISSEMENT,1.0,b'75108',b'408',b'N',NaN,1987.0,b'BARCOS*ROBERT/ ...
3,1,104,BERTRAND,DIDIER CHARLES,1911.0,8.0,14.0,78.0,0.0,Marly le Roi,...,78.0,NaN,MARLY-LE-ROI,1.0,b'78372',b'648',b'N',NaN,1988.0,b'BERTRAND*DIDIER CHARLES NOEL/ ...
4,1,105,BESSE,RENE,1906.0,3.0,1.0,75.0,0.0,Paris,...,75.0,NaN,PARIS 11,1.0,b'75111',b'1352',b'N',NaN,1985.0,b'BESSE*RENE ANDRE JACQUES/ ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,1,1029,ROHRBACHER,NICOLE MARIE THERESE,1929.0,3.0,18.0,95.0,0.0,Saint Leu-la-Forêt,...,78.0,NaN,SAINT-LEU-LA-FORET,2.0,b'78563',b'97',NaN,NaN,2022.0,b'ROHRBACHER*NICOLLE MARIE THERESE/ ...
947,1,1091,SIMON,ODETTE DENISE,1928.0,12.0,3.0,93.0,0.0,Bagnolet,...,75.0,NaN,BAGNOLET,2.0,b'75006',b'2878',NaN,NaN,2020.0,b'SIMON*ODETTE DENISE/ ...
948,2,1143,TRONQUA,MICHELLE PIERRETTE,1930.0,9.0,3.0,99.0,1.0,Aintemouchent,...,92.0,NaN,DEPARTEMENT D'ORAN,2.0,b'92352',b'183',NaN,NaN,2021.0,"b""TRONQUA*MICHELLE PIERRETTE/ ..."
949,1,1147,VAILLANT,GENEVIEVE GEORGETTE,1926.0,6.0,2.0,93.0,0.0,Pantin,...,75.0,NaN,PANTIN,2.0,b'75055',b'24',NaN,NaN,2013.0,b'VAILLANT*GENEVIEVE GEORGETTE/ ...


In [60]:
matching.to_csv('unique/uniques/matching.csv')

In [62]:
df = pd.read_csv('df_travail.csv')

liste = matching['identification'].to_list()
compteur = 0

for i in df.index.to_list():
  if df['identification'][i] in liste:
    compteur += 1

compteur

1070